In [1]:
import nivapy3 as nivapy
import pandas as pd

In [2]:
eng = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


# Tista fluxes of TOTN and TOTP

Tista utløp Femsjøen has quarterly monitoring from 2004 to 2016.

In [3]:
st_yr, end_yr = 2004, 2016
stn_id = 29830  # ØSTETIS (Tista utløp Femsjøen)
par_list = ["TOTN"]

In [4]:
# Get par ID of interest
par_df = nivapy.da.select_resa_station_parameters(
    [stn_id], f"{st_yr}-01-01", f"{end_yr}-12-31", eng
)
par_df = par_df.query("parameter_name in @par_list")
par_df["par_unit"] = par_df["parameter_name"] + "_" + par_df["unit"]
par_df

25 parameters available for the selected stations and dates.


,parameter_id,parameter_name,unit,par_unit
17,4,TOTN,µg/l N,TOTN_µg/l N


In [5]:
# Get chem
wc_df, dup_df = nivapy.da.select_resa_water_chemistry(
    [stn_id], par_df, f"{st_yr}-01-01", f"{end_yr}-12-31", eng
)
wc_df = wc_df[["sample_date"] + list(par_df["par_unit"])].set_index("sample_date")
wc_df.rename(
    {"TOTN_µg/l N": "TOTN_ug/l", "TOTP_µg/l P": "TOTP_ug/l"},
    axis="columns",
    inplace=True,
)
wc_df.head()

,TOTN_ug/l
sample_date,
2004-03-23,885
2004-05-10,975
2004-08-09,850
2004-10-04,845
2005-02-14,900


In [6]:
# Get flow
q_df = nivapy.da.extract_resa_discharge(
    stn_id, f"{st_yr}-01-01", f"{end_yr}-12-31", eng
)
q_df.head()

,flow_m3/s
date,
2004-01-01,29.303731
2004-01-02,27.919507
2004-01-03,26.609203
2004-01-04,25.385660
2004-01-05,24.271542


In [7]:
flux_df = nivapy.stats.estimate_fluxes(
    q_df, wc_df, base_freq="D", agg_freq="A", method="ospar_annual"
)
flux_df

,TOTN_kg
year,
2004,627116.834016
2005,517940.504610
2006,745274.715368
2007,693601.016044
2008,814748.572071
2009,527432.222974
2010,397913.982289
2011,624316.926454
2012,602003.895026


In [10]:
# Phil estimates a ~35% increase in TOTN concs moving downstream
flux_df2 = nivapy.stats.estimate_fluxes(
    q_df, 1.35 * wc_df, base_freq="D", agg_freq="A", method="ospar_annual"
)
flux_df2

,TOTN_kg
year,
2004,8.466077e+05
2005,6.992197e+05
2006,1.006121e+06
2007,9.363614e+05
2008,1.099911e+06
2009,7.120335e+05
2010,5.371839e+05
2011,8.428279e+05
2012,8.127053e+05


In [11]:
# Convert difference to tonnes
diff_df = (flux_df2 - flux_df) / 1000
diff_df.columns = [i.replace("kg", "tonnes") for i in inc_df.columns]
diff_df

,TOTN_tonnes
year,
2004,219.490892
2005,181.279177
2006,260.846150
2007,242.760356
2008,285.162000
2009,184.601278
2010,139.269894
2011,218.510924
2012,210.701363


In [12]:
diff_df.mean()

TOTN_tonnes    213.504052
dtype: float64